In [1]:
import os
import datetime as dt
import json 

if "src" not in os.listdir():
    os.chdir("../../../")


from src.lib.alpaca_paper import AlpacaTrader
from src.lib.alpaca_historical import AlpacaData

In [2]:
KEYS_PATH = "alpaca_config.json" # Path to Alpaca Keys in user system
alpaca = hist = AlpacaTrader(KEYS_PATH, is_paper=True)
alpaca_data = AlpacaData(KEYS_PATH)
#alpaca_data.get_bars("SPY", dt.datetime.now()-dt.timedelta(minutes=3), dt.datetime.now(), log=False)

In [3]:
#response = alpaca.get_account()
#alpaca.get_holdings(raw=True)
curr_price = alpaca.get_current_price("SPY")

https://paper-api.alpaca.markets/v2/stocks/SPY/snapshot


In [4]:
curr_price.content

b'Not Found'

In [5]:
holdings = {"SPY": 0.01}
responses = alpaca.update_holdings(holdings, cancel_orders=False)
print(responses)

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
for response in responses:
    print(response.content)

b''
b''
b''
b'{"id":"eed404de-3e54-4433-95d7-4a8154e9e9c7","client_order_id":"d8f048fb-2152-4e2f-ba33-6a159bc9964c","created_at":"2021-09-14T21:51:55.21608Z","updated_at":"2021-09-14T21:51:55.21608Z","submitted_at":"2021-09-14T21:51:55.208421Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b28f4066-5c6d-479b-a2af-85dc1a8f16fb","symbol":"SPY","asset_class":"us_equity","notional":"30000","qty":null,"filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"buy","time_in_force":"day","limit_price":null,"stop_price":null,"status":"accepted","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null}'


In [ ]:
orders = alpaca.get_orders(status="open")

In [ ]:
orders.content

b'[{"id":"eed404de-3e54-4433-95d7-4a8154e9e9c7","client_order_id":"d8f048fb-2152-4e2f-ba33-6a159bc9964c","created_at":"2021-09-14T21:51:55.21608Z","updated_at":"2021-09-14T21:51:55.314517Z","submitted_at":"2021-09-14T21:51:55.208421Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b28f4066-5c6d-479b-a2af-85dc1a8f16fb","symbol":"SPY","asset_class":"us_equity","notional":"30000","qty":null,"filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"buy","time_in_force":"day","limit_price":null,"stop_price":null,"status":"new","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null}]'